## Multi-agent reinforcement learning

<small>see sections 1.1 to 1.4 and 5.3 to 5.4 of [Multi-Agent Reinforcement Learning: Foundations and Modern Approaches](https://www.marl-book.com) for a more in-depth overview.</small>

So far we tackled the problem of learning an optimal policy for a single agent inside a MDP, we now shift our focus towards contexts in which there are _multiple_ agents acting in the MDP with (possibly) different goals.

A **multi-agent system** is composed of an environment and multiple decision-making agents, also called _players_, that interact in the environment to achieve certain goals. The environment evolves during play and the players refine their respective strategies, which can be collaborative, competitive or a mix of the two.

As an example, consider a fleet of autonomous vehicles with the goal of driving as quickly and safely as possible, a team of competing players in a videogame or a group of automated traders that manage each their own investments in a public market.

At the start of a round of the game, each player sees the state of the environment and picks an action, the round ends after every player has chosen an action, then the envoriment evolves accordingly and each players collects their reward to update their respective policy.

From the point of view of the agent little has changed, they still have a set of actions, a policy, an enviroment and the goal of optimizing their policy. The difference lies in the fact that **the immediate reward achieved by any player depends on the actions of the other players on the same round**.

### The goal of multi-agent reinforcement learning
In the classic RL case, a policy is optimal if it maximises the _return_, defined as the sum of the immediate reward obtained during play, in the multi-agent case, any player's return depends on the other players' actions and ultimately depends on their policies.

**If the return of a policy depends on the other policies, how do we define optimality?**